# Final project for course 02806 Social data analysis and visualization
> An analysis and visualization for novel Covid-19 virus

- toc: true 
- badges: true
- comments: false
- categories: [data_analysis, visualization]

# About

Some introduction here

<div class="flourish-embed" data-src="story/261608" data-url="https://flo.uri.sh/story/261608/embed">
    <script src="https://public.flourish.studio/resources/embed.js">
    </script>
</div>